#Topic Modelling su dati unsupervised

In [ ]:
import os
import numpy as np
import glob
import warnings
import nltk
import string
import collections
import pandas as pd
from shutil import copyfile

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Caricamento dei dati

In [ ]:
copyfile('/content/gdrive/MyDrive/Text Mining/TM_Project/unsup.csv', 'unsup.csv')


'unsup.csv'

In [ ]:
unsup=pd.read_csv('unsup.csv')

In [ ]:
unsup.rename(columns = {'0' : 'text'}, inplace = True)

In [ ]:
unsup

,Unnamed: 0,text
0,0,"['made', 'tv', 'pilot', 'miami', 'vice', 'cop'..."
1,1,"['movie', 'said', 'irish', 'english', 'swedish..."
2,2,"['movie', 'get', 'released', 'properly', 'watc..."
3,3,"['story', 'original', 'coherent', 'enjoyable',..."
4,4,"['fair', 'reasonable', 'movie', 'recluse', 'ex..."
...,...,...
49995,49995,"['well', 'always', 'bit', 'difficult', 'wester..."
49996,49996,"['hatchet', 'job', 'fully', 'intended', 'movie..."
49997,49997,"['salacious', 'white', 'water', 'raft', 'race'..."
49998,49998,"['good', 'movie', 'special', 'effect', 'really..."


In [ ]:
#Rimozione di una colonna non necessaria
unsup.drop(['Unnamed: 0'],axis=1, inplace=True)
unsup

,text
0,"['made', 'tv', 'pilot', 'miami', 'vice', 'cop'..."
1,"['movie', 'said', 'irish', 'english', 'swedish..."
2,"['movie', 'get', 'released', 'properly', 'watc..."
3,"['story', 'original', 'coherent', 'enjoyable',..."
4,"['fair', 'reasonable', 'movie', 'recluse', 'ex..."
...,...
49995,"['well', 'always', 'bit', 'difficult', 'wester..."
49996,"['hatchet', 'job', 'fully', 'intended', 'movie..."
49997,"['salacious', 'white', 'water', 'raft', 'race'..."
49998,"['good', 'movie', 'special', 'effect', 'really..."


Caricamento di alcune funzioni utili per il preprocessing

In [ ]:
!wget -q "https://www.dropbox.com/s/yfjx9nhrky1ifws/preprocessing_functions.py" -O preprocessing_functions.py

In [ ]:
 from preprocessing_functions import remove_punctuation, remove_numbers, remove_stopwords,remove_extra_whitespace

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
unsup.text = unsup.text.apply(remove_punctuation)
unsup.text = unsup.text.apply(remove_extra_whitespace)
unsup.text = unsup.text.apply(remove_numbers)

In [ ]:
# Don't print warnings
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [ ]:
unsup

,text
0,made tv pilot miami vice cop show made popular...
1,movie said irish english swedish wa enough mak...
2,movie get released properly watching extra dvd...
3,story original coherent enjoyable watch nothin...
4,fair reasonable movie recluse experiencing str...
...,...
49995,well always bit difficult western country get ...
49996,hatchet job fully intended movie found quiet t...
49997,salacious white water raft race actually water...
49998,good movie special effect really bad pay twent...


In [ ]:
!pip install -q pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 5.3 MB/s 


In [ ]:
from __future__ import print_function

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
texts = list(unsup.text.str.split())

In [ ]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=7, no_above=0.3)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

* Fit LDA model


In [ ]:
lda_gensim = LdaModel(corpus, num_topics=8)

In [ ]:
# Print the Keyword in the 8 topics
lda_gensim.print_topics()


In [ ]:
#Getting the words from the dictionary
dictionary[84] 

In [ ]:
doc_lda = lda_gensim[corpus]

In [ ]:
pyLDAvis.gensim.prepare(lda_gensim, corpus, dictionary, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1     -108.958252 -208.941483       1        1  26.142058
6      186.487061  -98.103249       2        1  15.236659
5       19.245832  160.858780       3        1  13.026248
3      170.658829   75.445076       4        1  10.989375
2       -1.386411  -35.043739       5        1   9.299168
7     -145.800095  106.944611       6        1   9.199381
0       64.834541 -221.825272       7        1   8.662472
4     -196.906036  -58.990913       8        1   7.444638, topic_info=       Term          Freq         Total Category  logprob  loglift
588     bad  19348.000000  19348.000000  Default  30.0000  30.0000
69     show  16525.000000  16525.000000  Default  29.0000  29.0000
207   funny   9127.000000   9127.000000  Default  28.0000  28.0000
443   great  18074.000000  18074.000000  Default  27.0000  27.0000
706  horror   7705.000000   7705.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
165   thing    919.650348  16122.597073   Topic8  -6.0612  -0.2663
518    back    847.725130  10165.713923   Topic8  -6.1426   0.1135
46     many    840.229003  13008.698129   Topic8  -6.1515  -0.1420
105   could    845.146864  15755.536317   Topic8  -6.1456  -0.3278
279    know    837.092309  13696.239734   Topic8  -6.1552  -0.1973

[795 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
6233       2  0.988606    aamir
4592       7  0.993780   abbott
3600       1  0.955734  abysmal
3600       2  0.012047  abysmal
3600       7  0.020078  abysmal
...      ...       ...      ...
7926       2  0.007835        ´
7926       3  0.976721        ´
7926       7  0.015669        ´
24035      4  0.994532        –
24036      4  0.995335        ’

[2857 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 6, 4, 3, 8, 1, 5])

Evaluation metrics

In [ ]:
print('Perplexity: ',
lda_gensim.log_perplexity(corpus))

Perplexity:  -8.417343548441622


In [ ]:
from gensim.models import CoherenceModel

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=texts, dictionary=dictionary, coherence='c_v')

In [ ]:
coherence_lda = coherence_model_lda.get_coherence()    
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.29392196015296723


In [ ]:
lda_5_topic = LdaModel(corpus, num_topics=5)
coherence_model_lda_5_topic = CoherenceModel(model=lda_5_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_5_topic = coherence_model_lda_5_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_5_topic)

Coherence Score:  0.2863608640713181


In [ ]:
lda_10_topic = LdaModel(corpus, num_topics=10)
coherence_model_lda_10_topic = CoherenceModel(model=lda_10_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_10_topic = coherence_model_lda_10_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_10_topic)

Coherence Score:  0.31871498929373293


In [ ]:
lda_15_topic = LdaModel(corpus, num_topics=15)
coherence_model_lda_15_topic = CoherenceModel(model=lda_15_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_15_topic = coherence_model_lda_15_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_15_topic)

Coherence Score:  0.33936971927795817


In [ ]:
lda_19_topic = LdaModel(corpus, num_topics=19)
coherence_model_lda_19_topic = CoherenceModel(model=lda_19_topic, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_19_topic = coherence_model_lda_19_topic.get_coherence()    
print('Coherence Score: ', coherence_lda_19_topic)

Coherence Score:  0.33756117190177964


In [ ]:
#pyLDAvis.gensim.prepare(lda_8_topic, corpus, dictionary, mds='tsne')

In [ ]:
#pyLDAvis.gensim.prepare(lda_5_topic, corpus, dictionary, mds='tsne')